In [16]:
import pandas as pd 

df = pd.read_excel("2022.10.29-30_Orders.xlsx")
print(df)

         id  service_area_id              order_no  trip_type  hail_type wav  \
0    280761             5915  20221030105413cnKVsH          2          1   f   
1    280580             5915  202210300500109nObYm          2          1   f   
2    280581             5917  20221030042430nFwmr4          2          1   f   
3    280582             5917  20221030045536AteV9o          2          2   f   
4    280583             5915  20221030045022YM2U1R          2          2   f   
..      ...              ...                   ...        ...        ...  ..   
395  280656             5917  20221030072121HGMVmg          2          1   f   
396  280653             5915  20221030072528apWTcN          2          1   f   
397  280655             5915  20221030071359JjUGOH          2          3   f   
398  280657             5917  20221030065758HZ3nad          2          1   f   
399  280658             5915  20221030072549YBuo9N          2          1   f   

    driver_username                    

In [17]:
def filter_driver(driver_username : str, df):
    return df[df['driver_username'] == driver_username]

def filter_order(order_status : int, df):
    return df[df['order_status'] == order_status]


COMPLETED_ORDER = 50
# Define drivers as unique driver user name
drivers = df["driver_username"]
drivers = drivers[~pd.isnull(drivers)] #～ == negation 看driver_username是不是 null， 得到drivers这个df
drivers = drivers.tolist() #把 drivers 这个df 变成list
messages = []


# for loop
for d in drivers: 
    total = filter_driver(d , df).shape[0] #总共多少单
    completed = filter_order(COMPLETED_ORDER ,filter_driver(d , df)).shape[0] #总共完成了多少单
    incompleted = total - completed #总共未完成多少单

    # print(total)
    # print(completed)
    # print(incompleted)

    text_messages = "您已完成{}单 还有{}单未完成".format(completed, incompleted) #csv里的 message string
    messages.append(text_messages)
    #print(messages)

text_messages_dictionary = {'person':drivers,'message': messages} #defined a dictionary 



# export pd.df to csv
df_tocsv = pd.DataFrame(text_messages_dictionary) #用text_message_dictionary 建立 df_tocsv这个df
df_tocsv = df_tocsv.drop_duplicates() #removed duplicates
df_tocsv.to_csv('result.csv', encoding='utf-8', index = False) #把df变成csv




In [18]:
#join a wechat id csv with correspondingdrivers and messages csv

df_wechat = pd.read_csv("wechat.csv") #把wechat.csv读成df


df_tocsv = pd.merge(df_wechat, df_tocsv, on = 'person', how = 'inner') #merge了df_wechat and df_tocsv

In [19]:
from pynput.mouse import Button, Controller as mou_cl  # 鼠标控制器
from pynput.keyboard import Key, Controller as key_cl  # 键盘控制器
from pynput.keyboard import Key, Controller
import sys
import os
import time

# 键盘的控制函数


def keyboard_input(string):
    # 获取键盘权限
    keyboard = key_cl()
    # 设置数据类型
    keyboard.type(string)


def mouse_click():
    # 获取鼠标权限
    mouse = mou_cl()
    # 模拟左键按下
    mouse.press(Button.left)
    # 模拟左键弹起
    mouse.release(Button.left)


# 实现消息的发送函数
def send_message(number, string):
    keyboard = key_cl()

    for i in range(number):
        keyboard_input(string)
        # mouse_click()
        time.sleep(0.4)
        keyboard.press(Key.enter)
        keyboard.release(Key.enter)
        time.sleep(0.4)


def check_message(file_name: str):
    file_path = os.path.dirname(os.path.abspath(__file__))
    file_path = file_path + '/' + file_name
    print(file_path)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        return df
    else:
        print("No 'message.csv' can be found, please create such csv (with header) that includes two column: person, message")
        sys.exit()


def find_person(keyboard):
    keyboard.press(Key.cmd.value)
    keyboard.press('f')
    keyboard.release('f')
    keyboard.release(Key.cmd.value)


def send_message_one_row(row, keyboard):
    find_person(keyboard)

    send_message(1, row['wechat_id'])
    time.sleep(1)
    send_message(1, row['message'])
    time.sleep(0.5)
    keyboard.press(Key.enter)
    keyboard.release(Key.enter)
    time.sleep(1)


def send_messages(df):
    keyboard = Controller()
    for index, row in df.iterrows():
        send_message_one_row(row, keyboard)




In [21]:

assert len(df_tocsv.columns) == 3
assert df_tocsv.columns[0] == 'person'
assert df_tocsv.columns[1] == 'wechat_id'
assert df_tocsv.columns[2] == 'message'
print("程序在3秒钟之后开始执行")
time.sleep(3)
send_messages(df_tocsv)


程序在3秒钟之后开始执行
